In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [ ]:
ProdDict = {}

ProdDict['ProductName'] = []
ProdDict['ProductDescription']=[]
ProdDict['ProductIngredients']=[]
ProdDict['ProductPrice']=[]
ProdDict['href']=[]

In [ ]:
UserDict = {}

UserDict['ProductName'] = []
UserDict['Username'] = []
UserDict['Reviews']=[]
UserDict['Ratings'] = []


In [ ]:
def appendData(product_name,review_all,user_all,rating_all):       

    UserDict['ProductName'].extend((product_name for i in range(len(review_all))))
    UserDict['Username'].extend((u.text.strip() for u in user_all))
    UserDict['Reviews'].extend((r.text.strip() for r in review_all))
    UserDict['Ratings'].extend((r['title'] for r in rating_all))
    
def retrieve_reviews_user(nextpage,product_name):   
    
    rating_all=nextpage.find_all('abbr',class_="bv-rating bv-rating-stars bv-rating-stars-off")
    user_all=nextpage.find_all('span',class_='bv-author')
    review_all=nextpage.find_all('div',class_='bv-content-summary-body-text') 
    appendData(product_name,review_all,user_all,rating_all)
    
def extract_info(nextpage,tag,classname):
    try:
        html = nextpage.find(tag, class_=classname)        
        if html:
            return html.text.strip()            
        else:
            return "Error while retreiving html"
    except AttributeError:
        return None

In [ ]:
def main(products_html,driver):
    for p in products_html:
        product_name=p.find('div',class_="product__title").text.strip()
        product_price=p.find('div',class_="product__price").text.strip()
        href=p.get("href")

        ProdDict['ProductName'].append(product_name)
        ProdDict['ProductPrice'].append(product_price)    
        ProdDict['href'].append(href)
        
        driver.get(href)
        nextpage=BeautifulSoup(driver.page_source,'lxml')  
        prod_desc = nextpage.find_all('div', class_="details",itemprop="description")[2].text.strip()
        
        ProdDict['ProductDescription'].append(prod_desc)
        prod_ing = extract_info(nextpage,'section', "product-info-section ingredients")        
        ProdDict['ProductIngredients'].append(prod_ing)
       
        
        retrieve_reviews_user(nextpage,product_name)

        while nextpage:
            try:
                nextpage_link = nextpage.find('a', class_="bv-content-btn bv-content-btn-pages bv-content-btn-pages-last bv-focusable bv-content-btn-pages-active")
                if nextpage_link:
                    nextpage_url = nextpage_link.get("href")
                    driver.get(nextpage_url)
                    nextpage = BeautifulSoup(driver.page_source, 'lxml')
                    retrieve_reviews_user(nextpage,product_name)
                else:
                    nextpage = None
            except AttributeError:
                retrieve_reviews_user(nextpage,product_name)
                nextpage = None

In [ ]:
def next_page(driver, times):
    for _ in range(times-1):
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, ".pager__button--next"))
        )
        next_button.click()

In [ ]:

def scrapeData(page,initial_page_url,UserDataFileName,ProductDataFileName):

    driver = Chrome()
   
    driver.get(initial_page_url)

    next_page(driver,page)

    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "category-product"))
    )


    soup=BeautifulSoup(driver.page_source,'lxml')    
    products_html=soup.find_all('a',class_="category-product")


    main(products_html,driver)
    # converting the dictionary to dataframe:
    ProductData=pd.DataFrame(ProdDict)
    UserData=pd.DataFrame(UserDict)

    UserData.to_csv(f'{UserDataFileName}_{page}.csv')
    ProductData.to_csv(f'{ProductDataFileName}_{page}.csv')

In [ ]:
scrapeData(1,"https://www.chemistwarehouse.com.au/shop-online/2351/la-roche-posay","La Roche-Posay_userData_ProductsPage","La Roche-Posay_Products_InfoPage")